# Function Calling

This is a simple example of how to enable agents to use functions.

In [2]:
# First, import the agent_callable decorator from botplayers.
from botplayers import agent_callable, Agent

In [3]:
# Then, define a function that can be called by agents.
# 1. The function must be decorated with agent_callable.
# 2. The function can have any number of arguments.
# 3. The function can return any JSON-serializable object.
# 4. [IMPORTANT] Tell agents how to call this function by writing a docstring.
@agent_callable()
def calculator(python_math_expression: str):
    """
    A simple calculator that can do basic math operations.
    Use math module to call functions or use constants like 
        math.sqrt, math.sin, math.cos, math.tan, math.asin, math.acos, math.atan, 
        math.log, math.log10, math.exp, math.e, math.pi.

    Args:
        python_math_expression: a python math expression.
    """
    import math
    return {'result': eval(python_math_expression, {'math': math})}

In [4]:
# Now, let's see how to call this function from agents.
# First, create an agent. Use prompt to tell agents what they can do.
agent = Agent(
    name='Bot', 
    prompt="You are a helpful bot. You can use functions to accomplish tasks.",
    function_call_repeats=10, 
    ignore_none_function_messages=False)

In [5]:
# Then, run the agent.
user_message = "Could please tell me what is 234*(212-3456)/(121-21241)?"
agent.receive_message({'role': 'user', 'content': user_message})
agent.think_and_act()

Bot received a message: Could please tell me what is 234*(212-3456)/(121-21241)?
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': '234*(212-3456)/(121-21241)'}
        response: {'result': 35.94204545454546}
Bot >> 
The result of the expression 234*(212-3456)/(121-21241) is approximately 35.94204545454546.


# World Interaction

Agents can also use functions to interact with a stateful environment, e.g the world.

In [6]:
# Now, lets import the World class from the botplayers package
from botplayers import World

# Define simple environment with a simple state.
class Env(World):
    messages: dict[str, list] = dict()

    # Define a callable classmethod that can be called by agents.
    # Note that, the `self` and the `agent_name` parameters are invisible to agents.
    # The `agent_name` parameter is automatically filled by the agent's name.
    # The `self` parameter is automatically filled by the world instance.
    # Therefore, do not mention `self` and `agent_name` in the docstring.
    @agent_callable()
    def store(self, agent_name: str, message: str):
        """
        Store a message from an agent.

        Args:
            message: the message to store.
        """
        if agent_name not in self.messages:
            self.messages[agent_name] = []
        self.messages[agent_name].append(message)
        return 'done'

    @agent_callable()
    def get_stored_messages(self, agent_name):
        """
        Get all stored messages.

        Returns:
            messages: the list of messages.
        """
        return {'messages': self.messages.get(agent_name, [])}

In [7]:
# Now lets create an instance of Env.
env = Env()

# Let the agent interact with the environment.
# Note that botplayers supports error feedback, so that agents can learn from their mistakes.
user_message = 'Please propose three complex math calculation problems. First compute them, then store both the questions and answers.'
agent.receive_message({'role': 'user', 'content': user_message})
agent.think_and_act(env)

user_message = 'Please show me the questions and answers.'
agent.receive_message({'role': 'user', 'content': user_message})
agent.think_and_act(env)

Bot received a message: Please propose three complex math calculation problems. First compute them, then store both the questions and answers.
Bot >> 


    Bot is calling function calculator ...
        with arguments {'python_math_expression': '2**3 + 7**2'}
        response: {'result': 57}
Bot >> 
    Bot is calling function store ...
        with arguments {'message': 'What is the result of 2**3 + 7**2?'}
        response: done
Bot >> 
    Bot is calling function store ...
        with arguments {'message': 'The result of 2**3 + 7**2 is 57.'}
        response: done
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': 'sqrt(121) + log10(1000)'}
        error: name 'sqrt' is not defined
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': 'import math\nmath.sqrt(121) + math.log10(1000)'}
        error: invalid syntax (<string>, line 1)
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': 'math.sqrt(121) + math.log10(1000)'}
        response: {'result': 14.0}
Bot >> 
    Bot is calling functio